# datasets-R

In [1]:
import pandas as pd

In [2]:
# http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_html.html
# pip install html5lib
url = "http://d.hatena.ne.jp/hoxo_m/20120214/p1"
dfs = pd.read_html(url, header=0)
type(dfs), len(dfs), type(dfs[0])

(list, 2, pandas.core.frame.DataFrame)

In [4]:
dnames = dfs[0]
dnames.tail()

データセット名               タイトル  \
23              USArrests        合州国の州別暴力犯罪率   
24         USJudgeRatings  弁護士による合州国最高裁判事の評価   
25  USPersonalExpenditure            個人消費データ   
26               VADeaths             死亡率データ   
27                  women     米国女性の平均身長と平均体重   

                                                   説明  詳細  DL  
23  このデータセットは 1973 年の合州国の 50 の州毎の、住民10万人あたりの暴行、殺人、...  詳細  DL  
24                                  弁護士による合州国最高裁判事の評価  詳細  DL  
25  このデータセットは、1940,1945, 1950 そして 1960 年における、 次の各項...  詳細  DL  
26                         1940年代のバージニア州の100人あたりの死亡率。  詳細  DL  
27                          ３０歳から３９歳のアメリカ人女性の平均身長と体重。  詳細  DL

In [5]:
#  pip install pyquery
# Successfully installed cssselect-0.9.1 pyquery-1.2.13
# http://pyquery.readthedocs.io/en/latest/attributes.html
import pyquery

d = pyquery.PyQuery(url)
# Chrome で copy as selector
# #days > div > div > div.section > table:nth-child(15) > tbody > tr:nth-child(1) > th:nth-child(4)
dnames["詳細_URL"] = pd.Series(pyquery.PyQuery(a).attr('href') for a in d.find("table:first tr td:nth-child(4) > a"))
dnames["DL_URL"] = pd.Series(pyquery.PyQuery(a).attr('href') for a in d.find("table:first tr td:nth-child(5) > a"))
dnames.tail()

データセット名               タイトル  \
23              USArrests        合州国の州別暴力犯罪率   
24         USJudgeRatings  弁護士による合州国最高裁判事の評価   
25  USPersonalExpenditure            個人消費データ   
26               VADeaths             死亡率データ   
27                  women     米国女性の平均身長と平均体重   

                                                   説明  詳細  DL  \
23  このデータセットは 1973 年の合州国の 50 の州毎の、住民10万人あたりの暴行、殺人、...  詳細  DL   
24                                  弁護士による合州国最高裁判事の評価  詳細  DL   
25  このデータセットは、1940,1945, 1950 そして 1960 年における、 次の各項...  詳細  DL   
26                         1940年代のバージニア州の100人あたりの死亡率。  詳細  DL   
27                          ３０歳から３９歳のアメリカ人女性の平均身長と体重。  詳細  DL   

                                               詳細_URL  \
23  http://www.is.titech.ac.jp/~mase/mase/html.jp/...   
24  http://www.is.titech.ac.jp/~mase/mase/html.jp/...   
25  http://www.is.titech.ac.jp/~mase/mase/html.jp/...   
26  http://www.is.titech.ac.jp/~mase/mase/html.jp/...   
27  http://www.is.titech.ac.jp/~mase/mase/html.jp/...   

                                               DL_URL  
23  http://dl.dropbox.com/u/432512/20120210/data/U...  
24  http://dl.dropbox.com/u/432512/20120210/data/U...  
25  http://dl.dropbox.com/u/432512/20120210/data/U...  
26  http://dl.dropbox.com/u/432512/20120210/data/V...  
27  http://dl.dropbox.com/u/432512/20120210/data/w...

In [6]:
d, d("table"), type(d[0])

([<html>], [<table>, <table>], lxml.html.HtmlElement)

In [7]:
methods = pd.read_html("http://d.hatena.ne.jp/hoxo_m/20120214/p1")[1]
methods

0                                                  1
0     回帰  anscombe, attitude, cars, esoph, Formaldehyde,...
1   分散分析  HairEyeColor, infert, InsectSprays, PlantGrowt...
2  多変量解析  airquality, attenu, attitude, eurodist, iris, ...
3  時系列解析                 airmiles, co2, discoveries, infert

In [8]:
methods.columns = ["method", "dataset_name"]
method_expand = methods.dataset_name.str.replace(", ", ",").str.split(",", expand=True)
method_expand

0         1             2            3             4  \
0      anscombe  attitude          cars        esoph  Formaldehyde   
1  HairEyeColor    infert  InsectSprays  PlantGrowth       Titanic   
2    airquality    attenu      attitude     eurodist          iris   
3      airmiles       co2   discoveries       infert          None   

                  5                 6          7               8      9  
0              iris  LifeCycleSavings    Titanic     ToothGrowth  women  
1       ToothGrowth     UCBAdmissions   VADeaths            None   None  
2  LifeCycleSavings     OrchardSprays  USArrests  USJudgeRatings   None  
3              None              None       None            None   None

In [9]:
melted = pd.melt(
    pd.concat([methods[["method"]], method_expand], axis=1),
    id_vars="method")
melted.tail()

method variable  value
35  時系列解析        8   None
36     回帰        9  women
37   分散分析        9   None
38  多変量解析        9   None
39  時系列解析        9   None

In [10]:
melted = melted.dropna()
melted["lower_value"] = melted.value.str.lower()
method_data_map = melted.sort_values(["method", "lower_value"])[["method", "value"]].reset_index(drop=True)
method_data_map.tail()

method           value
26  多変量解析  USJudgeRatings
27  時系列解析        airmiles
28  時系列解析             co2
29  時系列解析     discoveries
30  時系列解析          infert

In [11]:
method_data_map["適用可能"] = "◯"
method_data_map_pivoted = method_data_map.pivot(index="value", columns="method", values="適用可能").fillna("☓").reset_index()
method_data_map_pivoted.head()

method             value 分散分析 回帰 多変量解析 時系列解析
0           Formaldehyde    ☓  ◯     ☓     ☓
1           HairEyeColor    ◯  ☓     ☓     ☓
2           InsectSprays    ◯  ☓     ☓     ☓
3       LifeCycleSavings    ☓  ◯     ◯     ☓
4          OrchardSprays    ☓  ☓     ◯     ☓

In [117]:
dnames_detail = dnames.merge(method_data_map_pivoted, left_on="データセット名", right_on="value", how="outer")
dnames_detail

データセット名                            タイトル  \
0                airmiles                     商用航空会社マイレージ   
1              airquality                  ニューヨークの大気状態観測値   
2                anscombe  ``同じ'' 線形単回帰に対する Anscombe の四つ組   
3                  attenu         Joyner-Boore の地震波の減衰データ   
4                attitude                       管理者に対する態度   
5                    cars                          車の停車距離   
6                     co2         Mauna Loa 火山の大気中の炭酸ガス濃度   
7             discoveries                         重要な発見の数   
8                   esoph                   喫煙、アルコールと食道ガン   
9                    euro                     ヨーロッパの為替レート   
10               eurodist                    ヨーロッパの都市間の距離   
11           Formaldehyde                     ホルムアルデヒドの定量   
12           HairEyeColor            統計の講義を受講している学生の髪と瞳の色   
13                 infert                    自然・人工流産後の不妊症   
14           InsectSprays                     昆虫への薬剤噴霧の効果   
15                   iris         Edgar Anderson のあやめのデータ   
16                islands                     世界の主要な陸地の面積   
17       LifeCycleSavings                    各国の世代毎の貯蓄データ   
18          OrchardSprays                     果樹園への散布液の効果   
19            PlantGrowth                  植物の成長に関する実験の結果   
20                Titanic                    タイタニック号乗客の生存   
21            ToothGrowth       ギニアピッグの歯の成長に対するビタミン C の効果   
22          UCBAdmissions           カリフォルニア大学バークレイ分校の学生入学   
23              USArrests                     合州国の州別暴力犯罪率   
24         USJudgeRatings               弁護士による合州国最高裁判事の評価   
25  USPersonalExpenditure                         個人消費データ   
26               VADeaths                          死亡率データ   
27                  women                  米国女性の平均身長と平均体重   

                                                   説明  詳細  DL  \
0              1937年から1960年の各年の、合州国の商用航空会社の課税利用者マイル数。  詳細  DL   
1                         ニューヨークの大気状態観測値。1973 年の五月から。  詳細  DL   
2   同じ通常の統計的性質(平均、分散、相関、回帰直線)を持つが、全く異なる 四つの x-y デー...  詳細  DL   
3   このデータはカリフォルニア州の 23 の地震のピーク時加速度を、様々な観測基地で測定したデー...  詳細  DL   
4                                             （まだ）無い。  詳細  DL   
5       車が停車するまでに必要な距離のデータ。 データは 1920 年代に得られたことを注意せよ。  詳細  DL   
6   大気中の CO2 濃度が百万分の一単位 (ppm) で表され、preliminary 199...  詳細  DL   
7             1860年から1959年の各年における ``偉大な'' 発明と科学的発見の数。  詳細  DL   
8             フランスの Ile-et-Vilaine における食道ガンの類別研究のデータ。  詳細  DL   
9                                   様々なヨーロッパの通貨の交換比率。  詳細  DL   
10  このデータはヨーロッパの21の都市間の道路距離(km 単位)を与える。 データは ``The...  詳細  DL   
11  これらのデータは、クロム酸と濃縮硫酸を加えた結果生ずる紫色を分光計で 読みとるホルムアルデヒ...  詳細  DL   
12                 統計の講義を受講している 592 人の学生の髪、瞳の色と性別の分布。  詳細  DL   
13             これは条件付きロジスティック回帰が登場する前に行われた対応対照群研究である。  詳細  DL   
14                          異なる農薬を散布した農業実験単位毎の昆虫の計数値。  詳細  DL   
15  この有名な(Fiher もしくは Anderson の)あやめのデータセットは、三種類のあや...  詳細  DL   
16                    10,000 平方マイルを越える陸地の千平方マイル単位の面積。  詳細  DL   
17                                1960 - 1970 の貯蓄データ。  詳細  DL   
18  果樹園への散布液の様々な成分が 蜜蜂を忌避する効果を確かめる実験が、ラテン方陣デザインを用い...  詳細  DL   
19   対照群と二つの異なった処理条件のもとで得られた、収穫量（乾燥重量で計った）を比較する実験の結果。  詳細  DL   
20  このデータセットは大洋定期船 `Titanic' の破滅的な処女航海の乗客の運命に付いての情...  詳細  DL   
21  三種類のビタミン C の投与量(0.5, 1, そして 2mg)、二種類の 摂取法(オレンジ...  詳細  DL   
22  1973年のバークレイ校大学院の六つの最大学部の受験生の総合的データで、入学状況と性別で分類...  詳細  DL   
23  このデータセットは 1973 年の合州国の 50 の州毎の、住民10万人あたりの暴行、殺人、...  詳細  DL   
24                                  弁護士による合州国最高裁判事の評価  詳細  DL   
25  このデータセットは、1940,1945, 1950 そして 1960 年における、 次の各項...  詳細  DL   
26                         1940年代のバージニア州の100人あたりの死亡率。  詳細  DL   
27                          ３０歳から３９歳のアメリカ人女性の平均身長と体重。  詳細  DL   

                                               詳細_URL  \
0   http://www.is.titech.ac.jp/~mase/mase/html.jp/...   
1   http://www.is.titech.ac.jp/~mase/mase/html.jp/...   
2   http://www.is.titech.ac.jp/~mase/mase/html.jp/...   
3   http://www.is.titech.ac.jp/~mase/mase/html.jp/...   
4   http://www.is.titech.ac.jp/~mase/mase/html.jp/...   
5   http://www.is.ti